In [1]:
import os
os.getcwd()

'/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project/research'

In [2]:
os.chdir("../.")
os.getcwd()

'/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationconfig:
    root_dir : Path
    data_path : Path
    tokenizer_name : Path

In [4]:
from textsummarizer.constant import *
from textsummarizer.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationconfig:
        config = self.config.data_transformation

        data_transformation_config = DataTransformationconfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        return data_transformation_config

In [6]:
from textsummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [8]:
class DataTransformation:
    def __init__(self,config:DataTransformationconfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'],max_length=128,truncation=True)

            return {
                "input_ids" : input_encoding['input_ids'],
                "attention_mask": input_encoding["attention_mask"],
                "labels" : target_encoding['input_ids']
            }
        
    def convert(self):
        datasets = load_from_disk(self.config.data_path)
        datasets_pt = datasets.map(self.convert_examples_to_features,batched=True)
        datasets_pt.save_to_disk(os.path.join(self.config.root_dir,"dataset"))

In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-01-21 15:00:44,341: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-01-21 15:00:44,352: INFO: common: yaml file:params.yaml loaded successfully]
[2024-01-21 15:00:44,352: INFO: common: directory:artifacts already exists]


tokenizer_config.json: 100%|██████████| 88.0/88.0 [00:00<00:00, 134kB/s]
config.json: 100%|██████████| 1.12k/1.12k [00:00<00:00, 4.89MB/s]
spiece.model: 100%|██████████| 1.91M/1.91M [00:01<00:00, 1.42MB/s]
special_tokens_map.json: 100%|██████████| 65.0/65.0 [00:00<00:00, 252kB/s]
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/nehaagarwal/Desktop/projects/Text_Summarizer_Project/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 174460.52 examples/s]
